# Image Transformer
1. Image 투명화 
2. 투명 처리된 Image 저장
3. Image 정보(Category, Boundary)를 csv에 저장

In [10]:
import Img_Maker
# 이미지 파일 경로 입력 (메모리 최소 800MB 필요)

# 1 : 이미지 저장
# 0 : 패스
# 2 : 종료

# JSON File Path 설정
FILE_PATH = r'/home/lab10/JJC/EasyFree-Backend/IMAGE/Final_Model/3.json'
Img_Maker.check_img(FILE_PATH = FILE_PATH, save_pos = 476)

In [ ]:
# Image to Trans Data

In [2]:
import numpy as np
from PIL import Image
import Img_Maker
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('image_label.ipynb'))))
from PACK import FileSearch

In [3]:
def img2trans(FILE_PATH, count):
    img = Image.open(FILE_PATH)
    img = img.resize((224,224))
    png_data, label_map = Img_Maker.image_trans(np.array(img), 200)
    label_img = np.asarray(label_map).reshape(224,224,-1)
    file_name = FILE_PATH.split('/')[-1].split('_')[0].replace('2','')
    Img_Maker.save_img(png_data, './Target_Trans/%s'%(file_name+str(count)))
    Img_Maker.save_coco('./Target_Images/coco_target.csv', '%s'%(file_name+str(count)), file_name, 0, Img_Maker.find_boundary(label_img))


In [4]:
fl = FileSearch.search('/home/lab10/JJC/EasyFree-Backend/IMAGE/Target_Images', 'png')

In [5]:
i = 0 
for f in fl:
    img2trans(f, i)
    i+=1

In [38]:
# from matplotlib import pyplot as plt
# plt.imshow(np.asarray(png_data).reshape((224,224,-1)))



# Image Augmentation

In [58]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('/home/lab10/JJC/EasyFree-Backend/IMAGE/Trans_Image/Final_Model_lemon_2.png')  # PIL 이미지
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='Trans_Image', save_prefix='lemon2', save_format='png'):
    i += 1
    if i > 20:
        break  # 이미지 20장을 생성하고 마칩니다

# 참고자료
https://studyforus.com/innisfree/594134

In [11]:
import pandas as pd

In [13]:
pd.read_csv('/home/lab10/JJC/EasyFree-Backend/IMAGE/coco_file.csv')['category'].unique()

array([6000095799, 6000095829, 6000095904, 6000096175, 6000096206,
       6000096294, 6000096220])

# 학습 데이터 생성기 (이미지 합치기)
1. 매장 이미지(1024x1024)에 투명화한 상품 합치기
2. 매장 이미지 별 BOX 정보 COCO 파일 저장

In [6]:
from PIL import Image
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('image_label.ipynb'))))
from PACK import FileSearch

In [7]:
bg_fl = FileSearch.search('./Emart_Image', '.jpg')

In [9]:
for path in bg_fl:
    temp_bg = Image.open(path)
    x, y = temp_bg.size
    if x < 512 or y < 512:
        os.remove(path)

In [1]:
import Img_Maker
FILE_PATH = '/home/lab10/JJC/EasyFree-Backend/IMAGE/Target_Images/coco_target.csv'

# et_a 는 학습용 이미지 파일이름으로 돌릴때마다 바꿔줘야함 (안그러면 중복됨)
# coco_file.csv 파일 경로, 학습 이미지 파일 이름, 생성할 파일 개수
Img_Maker.mk_TrainData(FILE_PATH, 'et_b', 10000)

In [28]:
# 정확한 전처리는 학습시킬 때 확인